In [57]:
import pymysql
import pandas as pd
import shap as shap
import matplotlib.pyplot as plt


Get Data

In [58]:
# Set up the connection details
hostname = 'localhost'
username = 'root'
password = 'password'
database = 'economics_of_sports'

# Create a connection
connection = pymysql.connect(host=hostname,
                             user=username,
                             password=password,
                             db=database)

# Create a cursor
cursor = connection.cursor()

query = '''
select p.ws_id, tmvr.market_value, p.name, p.nationality, p.continent,  p.heigt , tmvr.player_age, p.nationality = l.country , tmvr.season,
    IF(LOCATE(' (', p.posistion) = 0, p.posistion, LEFT(p.posistion, LOCATE(' (', p.posistion) - 1)) AS posistion,
       ws_nl.tournament, ws_nl.elo_score, ws_nl.rating, ws_nl.minutes_played, ws_nl.motm_awards
        #,ws_ec.tournament, ws_ec.elo_score, ws_ec.rating, ws_ec.minutes_played, ws_ec.motm_awards
        #,ws_nl.tournament, ws_ntc.elo_score, ws_ntc.rating, ws_ntc.minutes_played, ws_ntc.motm_awards
        #,ws_nc.tournament, ws_nc.elo_score, ws_nc.rating, ws_nc.minutes_played, ws_nc.motm_awards
from player p
left join tm_market_value_raw tmvr on p.tm_id = tmvr.tm_player_id
    left join team t on tmvr.team_name = t.name
left join league l on t.league_id = l.id
left join (
    select * from ws_season
             where type = 'National Leagues'
) as ws_nl on ws_nl.ws_player_id = p.ws_id and ws_nl.season = tmvr.season
left join (
    select * from ws_season
             where type = 'European Cups'
) as ws_ec on ws_ec.ws_player_id = p.ws_id and ws_ec.season = tmvr.season
left join (
    select * from ws_season
             where type = 'National Team Cups'
) as ws_ntc on ws_ntc.ws_player_id = p.ws_id and ws_ntc.season = tmvr.season
left join (
    select * from ws_season
             where type = 'National Cups'
) as ws_nc on ws_nc.ws_player_id = p.ws_id and ws_nc.season = tmvr.season
where ws_nl.season is not null
group by posistion, ws_nl.tournament, tmvr.market_value, p.name, p.nationality, p.continent, p.heigt, tmvr.player_age, p.nationality = l.country, tmvr.season, p.ws_id, ws_nl.elo_score, ws_nl.rating, ws_nl.minutes_played, ws_nl.motm_awards, ws_ec.tournament, ws_ec.elo_score, ws_ec.rating, ws_ec.minutes_played, ws_ec.motm_awards, ws_nl.tournament, ws_ntc.elo_score, ws_ntc.rating, ws_ntc.minutes_played, ws_ntc.motm_awards, ws_nc.tournament, ws_nc.elo_score, ws_nc.rating, ws_nc.minutes_played, ws_nc.motm_awards
order by name, tmvr.season'''

# Execute a query to select all from the "employees" table
cursor.execute(query=query)

# Fetch all the rows
rows = cursor.fetchall()

# Get column names
column_names = [i[0] for i in cursor.description]

# Convert the data to a pandas DataFrame
df = pd.DataFrame(rows, columns=column_names)

# Close the cursor and connection
cursor.close()
connection.close()

Clean up data and split

In [59]:
df = df.dropna()
targets=df.market_value
df[["rating"]] = df[["rating"]].apply(pd.to_numeric)

inputs=df.drop(['ws_id', 'name', 'tournament', 'market_value'],axis='columns')

inputs = pd.get_dummies(inputs)

from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.25, random_state=3)

Create train model

In [60]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor()
reg.fit(X_train, y_train)
train_sq = reg.score(X_train, y_train)
r_sq = reg.score(X_test, y_test)
print(r_sq)


0.701514943323311


In [61]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters
param_grid = {
    'n_estimators': [10, 50, 100, 200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [2,4,6,8,10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
}

reg = RandomForestRegressor()

grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# print the best parameters
print("Best Parameters: ", grid_search.best_params_)

# calculate the training and testing score using the best model
best_grid = grid_search.best_estimator_

train_sq = best_grid.score(X_train, y_train)
r_sq = best_grid.score(X_test, y_test)

print("Training score: ", train_sq)
print("Testing score: ", r_sq)

Best Parameters:  {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Training score:  0.6798960232873827
Testing score:  0.5510948884373894


Analyse model

In [62]:
# Get feature importances
importances = reg.feature_importances_

# Create a pandas DataFrame with feature importances
importance_df = pd.DataFrame({"Feature": X_train.columns, "Importance": importances})

# Sort the DataFrame by importances
importance_df = importance_df.sort_values(by="Importance", ascending=False)

# Summarize feature importances
for i, row in importance_df.iterrows():
    print(f'Feature {row["Feature"]}: {row["Importance"]}')

# Plot feature importances
plt.barh(importance_df["Feature"], importance_df["Importance"])
plt.xlabel("Importance")
plt.ylabel("Features")
plt.title("Feature importances")
plt.gca().invert_yaxis()  # Invert the y-axis to show the most important at the top
plt.show()

NotFittedError: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.